## Bayesian Optimization Run

Inialize test function

In [ ]:
from alef.oracles.flexible_oracle import Flexible1DOracle
import numpy as np

# here we simulate a dataset - either you can plug in an own dataset or use the dataset classes of alef
# configure function that should be learned actively
# f = lambda x: 0.1*np.sin(30*x)+2*x-1.0
f = lambda x: (x - 0.75) * 10.0 * np.exp(-1 * np.power((x - 0.75) * 10.0, 2.0))
f = lambda x: x + 0.2 * np.sin(40 * x)

# Oracle configuration
test_function = Flexible1DOracle()
test_function.set_box_bounds(0.0, 1.0)
test_function.set_f(f)
test_function.plot()

Configurations of model and Bayesian optimizer

In [7]:
from alef.bayesian_optimization.bayesian_optimizer_factory import BayesianOptimizerFactory
from alef.configs.bayesian_optimization.bayesian_optimizer_configs import BOExpectedImprovementConfig, BOGPUCBConfig
from alef.configs.models.gp_model_config import (
    GPModelFastConfig,
)
from alef.configs.kernels.hhk_configs import HHKTwoLocalDefaultConfig

# Model configuration
kernel_config = HHKTwoLocalDefaultConfig(input_dimension=1)
model_config = GPModelFastConfig(kernel_config=kernel_config)
# model_config = BasicGPModelMarginalizedConfig(kernel_config=kernel_config)


bo_config_ei = BOExpectedImprovementConfig()
bo_config_gp_ucb = BOGPUCBConfig()
n_initial_datapoints = 5
n_bo_steps = 20
initial_data_seed = 10
do_plotting = True
FIT_GP_HYPERPARAMETER_AT_START = True
SET_INITIAL_LOCATIONS_MANUALLY = True
initial_xs = [0.1, 0.2, 0.15]

Initialize model and Bayesian optimizer with EI

In [8]:
from alef.models.gp_model import GPModel
from alef.models.model_factory import ModelFactory

model = ModelFactory.build(model_config)
if isinstance(model, GPModel) and FIT_GP_HYPERPARAMETER_AT_START:
    x_data, y_data = test_function.get_random_data(500)
    model.infer(x_data, y_data)
    model.fix_current_hyperparameters()

bayesian_optimizer = BayesianOptimizerFactory.build(bo_config_ei)
bayesian_optimizer.set_model(model)
bayesian_optimizer.set_oracle(test_function)
if SET_INITIAL_LOCATIONS_MANUALLY:
    initial_ys = []
    for initial_x in initial_xs:
        initial_y = test_function.query(initial_x)
        initial_ys.append(initial_y)
    initial_xs = np.expand_dims(np.array(initial_xs), axis=1)
    initial_ys = np.expand_dims(np.array(initial_ys), axis=1)
    bayesian_optimizer.set_train_set(initial_xs, initial_ys)
else:
    bayesian_optimizer.sample_train_set(n_initial_datapoints, initial_data_seed, set_seed=True)
bayesian_optimizer.set_do_plotting(do_plotting)

Run Bayesian optimizer with Expected Improvement (EI)

In [ ]:
validation_metrics_ei, _ = bayesian_optimizer.maximize(n_bo_steps)

Initialize same experiment but with GP-UCB

In [11]:
model = ModelFactory.build(model_config)
if isinstance(model, GPModel) and FIT_GP_HYPERPARAMETER_AT_START:
    x_data, y_data = test_function.get_random_data(500)
    model.infer(x_data, y_data)
    model.fix_current_hyperparameters()

bayesian_optimizer = BayesianOptimizerFactory.build(bo_config_gp_ucb)
bayesian_optimizer.set_model(model)
bayesian_optimizer.set_oracle(test_function)
if SET_INITIAL_LOCATIONS_MANUALLY:
    bayesian_optimizer.set_train_set(initial_xs, initial_ys)
else:
    bayesian_optimizer.sample_train_set(n_initial_datapoints, initial_data_seed, set_seed=True)
bayesian_optimizer.set_do_plotting(do_plotting)

Run BO with GP UCB

In [ ]:
validation_metrics_gp_ucb, _ = bayesian_optimizer.maximize(n_bo_steps)

Create learning curve plots

In [ ]:
from alef.utils.metric_curve_plotter import MetricCurvePlotter
import numpy as np

curve_plotter = MetricCurvePlotter(1)
x_axis = np.arange(0, n_bo_steps)
curve_plotter.add_metrics_curve(x_axis, validation_metrics_ei, "blue", "EI", 0)
curve_plotter.add_metrics_curve(x_axis, validation_metrics_gp_ucb, "red", "GP-UCB", 0)
curve_plotter.configure_axes(0, "Max Value", "Iteration", "Max Value", False, True)
curve_plotter.show()